<a href="https://colab.research.google.com/github/i40-Tools/I40KG-Embeddings/blob/master/Similarity_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# I40 standards landscape similarity analysis using embeddings

## Overview

In this notebook, we show the similarity analysis between Industry 4.0 Standards. 
To do so, we create embeddings about the Industry 4.0 Standards Knowledge Graph (I40KG) developed by [Grangel-Gonzales et. al.](https://www.researchgate.net/publication/318208930_The_Industry_40_Standards_Landscape_from_a_Semantic_Integration_Perspective)

The embeddings are located here: [I40 Embeddings](https://github.com/i40-Tools/I40KG-Embeddings/tree/master/logs_sto)

## Initial Configurations
First, let's import the required libraries to perform the similarity analysis.

In [0]:
# %ls

I40KG-Embeddings/  sample_data/


In [0]:
# !git clone https://github.com/i40-Tools/I40KG-Embeddings.git

Cloning into 'I40KG-Embeddings'...
remote: Enumerating objects: 85, done.
remote: Counting objects: 100% (85/85), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 85 (delta 36), reused 37 (delta 9), pack-reused 0
Unpacking objects: 100% (85/85), done.


In [0]:
import scipy
from scipy import spatial
import numpy as np
import math
import json
from scipy.spatial.distance import cdist

### Define function to print the result in tabular format 

In [0]:
#function to print result in table
def print_result(result):
    print ("{:<8}                                            {:<15}                         {:<10}".format('Framework A','Framework B','Score'))
    print ("----------------------------------------------------------------------------------------------------------------------")
    for key,value in result.items():
        val = str(value)
        val = val.strip("{}")
        val = val.strip("''")
        val = val.replace("':","     ")
        print("{:<8}       {:<15}".format(key +"      ", val))



### Define similarity function
We are going to us cosine distance to measure the similarity between two embeddings

In [0]:
#function to calculate cosine distance 
def cosine_similarity(vec1,vec2):
    sum11, sum12, sum22 = 0, 0, 0
    for i in range(len(vec1)):
        x = vec1[i]; y = vec2[i]
        sum11 += x*x
        sum22 += y*y
        sum12 += x*y
    return sum12/math.sqrt(sum11*sum22)

## Similarity among I40 Standarization Frameworks
In this section we show the analysis of similarity among standarization frameworks

In [0]:
!pip3 install rdflib

     |████████████████████████████████| 348kB 2.9MB/s 
     |████████████████████████████████| 51kB 19.1MB/s 


In [0]:
import json
from rdflib import Graph
import pprint

###Similarity among Frameworks
In this section we analyze the similarity of I4.0 frameworks at the high level, i.e., StandardizationFramework.


In [0]:
g = Graph()
g.parse("/content/I40KG-Embeddings/sto/sto.nt", format="nt")
    
len(g) # prints 2
 #check printing of the graph    
'''for stmt in g:
    pprint.pprint(stmt)'''

#query to get the framework/standard from the sto.nt file
#to get standards we have to replace sto:StandardizationFramework by sto:Standard in the query
#we can get standards of the frameworks as well by just changing the query

qres = g.query(
    """PREFIX owl: <http://www.w3.org/2002/07/owl#>
       PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
       PREFIX sto: <https://w3id.org/i40/sto#>
    
    select ?s where {
            ?s rdf:type sto:StandardizationFramework .
            } limit 1000""")
    
       
#to get the corresponding embeddings of the frameworks/standards from the json file 
with open("/content/I40KG-Embeddings/logs_sto/entities_to_embeddings.json",'rb') as f:
    array = json.load(f)
new_dict = {}
for row in qres:
    for key,value in array.items():
        if key == "%s" % row:
            tem = key
            new_dict[tem] = array[key] 
            print(new_dict)

#to put the frameworks/standards with their corresponding embeddings in a file            
with open('/content/I40KG-Embeddings/output_framework.json','w') as f:
    # this would place the entire output on one line
    # use json.dump(new_dict, f, indent=4) to "pretty-print" with four spaces per indent
    json.dump(new_dict, f)
    
    
#to read the file containing standards/frameworks along with their embeddings   
with open('/content/I40KG-Embeddings/output_framework.json', 'r') as f:
    array = json.load(f)
    
#compare each standard/framework with all the other standards/frameworks to find cosine similarity
result = {}
for key,value in array.items():
    temp,tempDict= 0,{}
    for keyC,valueC in array.items():
        if keyC == key:
            continue
        temp = scipy.spatial.distance.cosine(value,valueC) #send the values of the standards/frameworks to cosine similarity function
        tempDict[keyC] = temp
        val1 = min(tempDict, key=tempDict.get)
    res = {}
    res[val1] = tempDict[val1]
    #print (res)
    result[key]= res
    #result[key]= tempDict
        
#print the standards/frameworks along with their similar standards/frameworks and their similarity distance
print_result(result)

FileNotFoundError: ignored

### Similarity among Standards

In this section we analyze the similarity of I4.0 standards at the high level, i.e., Standard.

In [0]:
g = Graph()
g.parse("/content/I40KG-Embeddings/sto/sto.nt", format="nt")
    
len(g) # prints 2
 #check printing of the graph    
'''for stmt in g:
    pprint.pprint(stmt)'''

#query to get the framework/standard from the sto.nt file
#to get standards we have to replace sto:StandardizationFramework by sto:Standard in the query
#we can get standards of the frameworks as well by just changing the query

qres = g.query(
    """PREFIX owl: <http://www.w3.org/2002/07/owl#>
       PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
       PREFIX sto: <https://w3id.org/i40/sto#>
    
    select ?s where {
            ?s rdf:type sto:Standard .
            } limit 1000""")
    
       
#to get the corresponding embeddings of the frameworks/standards from the json file 
with open("/content/I40KG-Embeddings/logs_sto/entities_to_embeddings.json",'rb') as f:
    array = json.load(f)
new_dict = {}
for row in qres:
    for key,value in array.items():
        if key == "%s" % row:
            tem = key
            new_dict[tem] = array[key] 
    print(new_dict)

#to put the frameworks/standards with their corresponding embeddings in a file            
with open('/content/I40KG-Embeddings/output_standard.json','w') as f:
    # this would place the entire output on one line
    # use json.dump(new_dict, f, indent=4) to "pretty-print" with four spaces per indent
    json.dump(new_dict, f)
    
    
#to read the file containing standards/frameworks along with their embeddings   
with open('/content/I40KG-Embeddings/output_standard.json', 'r') as f:
    array = json.load(f)
    
#compare each standard/framework with all the other standards/frameworks to find cosine similarity
result = {}
for key,value in array.items():
    temp,tempDict= 0,{}
    for keyC,valueC in array.items():
        if keyC == key:
            continue
        temp = scipy.spatial.distance.cosine(value,valueC) #send the values of the standards/frameworks to cosine similarity function
        tempDict[keyC] = temp
        val1 = min(tempDict, key=tempDict.get)
    res = {}
    res[val1] = tempDict[val1]
    #print (res)
    result[key]= res
    #result[key]= tempDict
        
#print the standards/frameworks along with their similar standards/frameworks and their similarity distance
print_result(result)

FileNotFoundError: ignored

####Results
The following table shows the frameworks with high degree of similarites.

| Framework A | Framework B | Score |
| ----------------------|:---------------------:| ----------:|
| RAMI 4.0 |  IIRA  | 0.75 |


In [0]:
# TODO implement a fuction to print a table as the one above

###Similarity among different layers of the Standards
In this section we analyze the similarity among layers of different standards, e.g., RAMI 4.0 -> Layer A vs IIRA -> Layer X

In [0]:
# TODO implement a fuction that receives two frameworks e.g., RAMI4.0 , IIRA and as a result print a table as below

####Results
The following table shows the similarity among different layers of two frameworks.

| RAMI4.0 | IIRA | Score |
| --------------|:-------:| ----------:|
| Layer A |  Layer X  | 0.85 |
| Layer B |  Layer Y  | 0.75 |
| Layer C |  Layer Z  | 0.85 |


In [0]:
# TODO print a table like the above

##Similarity among Standards of the same Framework
In this section we show the analysis of similarity among standards belonging to the same framework.

In [0]:
import json
from rdflib import Graph
import pprint



g = Graph()
g.parse("/content/I40KG-Embeddings/sto/sto.nt", format="nt")
    
len(g) # prints 2
    
'''for stmt in g:
    pprint.pprint(stmt)'''
    
qres = g.query(
    """PREFIX owl: <http://www.w3.org/2002/07/owl#>
       PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
       PREFIX sto: <https://w3id.org/i40/sto#>
    
    select ?s where {
            ?s rdf:type sto:Standard .
            ?s sto:hasClassification ?c .
            ?c sto:isDescribedin <https://w3id.org/i40/sto#RAMI> .
            } limit 1000""")

'''    
with open("framework_entity.nt", "w") as fd:
    for row in qres:
        fd.write("%s" % row + "\n")
        #print("%s" % row)'''
        
#oldfile = open("framework_entity.nt", "r")
with open("/content/I40KG-Embeddings/logs_sto/entities_to_embeddings.json",'rb') as f:
    array = json.load(f)
new_dict = {}
for row in qres:
#for line in oldfile:
    #line1 = line.strip("\n")
    for key,value in array.items():
        if key == "%s" % row:
            tem = key
            new_dict[tem] = array[key] 
    print(new_dict)

with open('/content/I40KG-Embeddings/output_standard_same_framework.json','w') as f:
    # this would place the entire output on one line
    # use json.dump(lista_items, f, indent=4) to "pretty-print" with four spaces per indent
    json.dump(new_dict, f) 

    
#to read the file containing standards/frameworks along with their embeddings   
with open('/content/I40KG-Embeddings/output_standard_same_framework.json', 'r') as f:
    array = json.load(f)
    
#compare each standard/framework with all the other standards/frameworks to find cosine similarity
result = {}
for key,value in array.items():
    temp,tempDict= 0,{}
    for keyC,valueC in array.items():
        if keyC == key:
            continue
        temp = scipy.spatial.distance.cosine(value,valueC) #send the values of the standards/frameworks to cosine similarity function
        tempDict[keyC] = temp
        val1 = min(tempDict, key=tempDict.get)
    res = {}
    res[val1] = tempDict[val1]
    #print (res)
    result[key]= res
    #result[key]= tempDict
        
#print the standards/frameworks along with their similar standards/frameworks and their similarity distance
print_result(result)

##Similarity among Standards of different Framework
In this section we show the analysis of similarity among standards belonging to different frameworks.

In [0]:
import json
from rdflib import Graph
import pprint



g = Graph()
g.parse("C:/Users/Kaushikee/.spyder-py3/I40KG-Embeddings-master/sto/sto.nt", format="nt")
    
len(g) # prints 2
    
'''for stmt in g:
    pprint.pprint(stmt)'''
    
qres1 = g.query(
    """PREFIX owl: <http://www.w3.org/2002/07/owl#>
       PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
       PREFIX sto: <https://w3id.org/i40/sto#>
    
    select ?s where {
            ?s rdf:type sto:Standard .
            ?s sto:hasClassification ?c .
            ?c sto:isDescribedin <https://w3id.org/i40/sto#RAMI> .
            } limit 1000""")

qres2 = g.query(
    """PREFIX owl: <http://www.w3.org/2002/07/owl#>
       PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
       PREFIX sto: <https://w3id.org/i40/sto#>
    
    select ?s where {
            ?s rdf:type sto:Standard .
            ?s sto:hasClassification ?c .
            ?c sto:isDescribedin <https://w3id.org/i40/sto#ISA95> .
            } limit 1000""") 

'''    
with open("framework_entity.nt", "w") as fd:
    for row in qres:
        fd.write("%s" % row + "\n")
        #print("%s" % row)'''
        
#oldfile = open("framework_entity.nt", "r")
with open("C:/Users/Kaushikee/.spyder-py3/I40KG-Embeddings-master/logs_sto/entities_to_embeddings.json",'rb') as f:
    array = json.load(f)
new_dict = {}
for row in qres1:
#for line in oldfile:
    #line1 = line.strip("\n")
    for key,value in array.items():
        if key == "%s" % row:
            tem = key
            new_dict[tem] = array[key] 
    print(new_dict)
for row in qres2:
#for line in oldfile:
    #line1 = line.strip("\n")
    for key,value in array.items():
        if key == "%s" % row:
            tem = key
            new_dict[tem] = array[key] 
    print(new_dict)



with open('/content/I40KG-Embeddings/output_standard_different_framework.json','w') as f:
    # this would place the entire output on one line
    # use json.dump(lista_items, f, indent=4) to "pretty-print" with four spaces per indent
    json.dump(new_dict, f)
    
    
#to read the file containing standards/frameworks along with their embeddings   
with open('/content/I40KG-Embeddings/output_standard_different_framework.json', 'r') as f:
    array = json.load(f)
    
#compare each standard/framework with all the other standards/frameworks to find cosine similarity
result = {}
for key,value in array.items():
    temp,tempDict= 0,{}
    for keyC,valueC in array.items():
        if keyC == key:
            continue
        temp = scipy.spatial.distance.cosine(value,valueC) #send the values of the standards/frameworks to cosine similarity function
        tempDict[keyC] = temp
        val1 = min(tempDict, key=tempDict.get)
    res = {}
    res[val1] = tempDict[val1]
    #print (res)
    result[key]= res
    #result[key]= tempDict
        
#print the standards/frameworks along with their similar standards/frameworks and their similarity distance
print_result(result)


####Results
The following table shows the similarity among different layers of two frameworks.

| RAMI4.0 | IIRA | Score |
| --------------|:-------:| ----------:|
| Standard A |  Standard X  | 0.85 |
| Standard B |  Standard Y  | 0.75 |
| Standard C |  Standard Z  | 0.85 |
